In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from joblib import dump, load

In [21]:
df1 = pd.read_pickle("projects_16_18_essay_features_only.pkl")

In [3]:
df2 = pd.read_parquet("tokens_single_file_part-00000-5bdd9372-029e-4ce4-869e-12a302797c34-c000.snappy.parquet")

In [4]:
df3 = pd.read_pickle("other_feats.pkl")

In [9]:
df1.head()

,Project ID,Project Current Status,X_essay_len,X_essay_?!,X_essay_numbers
0,2d36bc7bb0e06fa27584712947b1e1ed,Fully Funded,1170,0,1
1,9e9172005e36c8ca6cf85839022cc430,Expired,1277,0,0
2,abce71a45bb063e40fba669d01d22c29,Fully Funded,2075,0,0
3,7150253bfc78d79c2b483ff16e2a552a,Fully Funded,1259,0,0
4,366c36725e9e7607b5c5e5c0eb7126b7,Fully Funded,1628,1,4


In [7]:
df2.head()

,ProjectID,student,school,learn,classroom,help,work,read,love,day,...,sport,hurricane,health,volleyball,basketball,soccer,college,museum,paint,activity
0,1dd5a2cc7141208616671df62172dfe2,5,1,0,1,0,3,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5f5d543c5840b3518b280cabd68a5a78,7,2,7,2,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,b85065c595a38b1ec7583799d0c220ef,12,8,2,1,1,1,2,4,0,...,0,0,0,0,0,0,0,0,0,0
3,1e0efc5f3bf4d1ba9e8a0722e4dbcd0e,8,1,3,1,0,0,6,3,0,...,0,0,0,0,0,0,0,0,0,0
4,e3eafe099ef343bc3ade232c4219572a,12,2,3,2,0,1,1,0,3,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df1.shape, df2.shape, df3.shape

((646531, 5), (646745, 86), (646517, 53))

In [6]:
df = df1.merge(df3, left_on = 'Project ID', right_on = 'Project ID').merge(df2, left_on = 'Project ID', right_on = 'ProjectID')
df = df[df['Project Current Status'] != 'Live']

In [7]:
df['Project Current Status'].value_counts()

Fully Funded    476709
Expired         127969
Name: Project Current Status, dtype: int64

In [8]:
df['Project Current Status Coded'] = df['Project Current Status'].apply(lambda x: int(x == 'Fully Funded'))

In [9]:
df['Project Current Status Coded'].value_counts(normalize=True)

1    0.788368
0    0.211632
Name: Project Current Status Coded, dtype: float64

In [10]:
del df['Project Current Status']
del df['ProjectID']
del df['Project ID']
df.shape

(604678, 141)

In [11]:
df.head()

,X_essay_len,X_essay_?!,X_essay_numbers,Project Valid Time,Project Cost,Teacher Project Posted Sequence,Project Year 2016,Project Year 2017,SUBCAT Literacy,SUBCAT Mathematics,...,hurricane,health,volleyball,basketball,soccer,college,museum,paint,activity,Project Current Status Coded
0,1170,0,1,120.0,509.89,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1277,0,0,119.0,284.27,16,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,2075,0,0,120.0,272.92,5,1,0,1,0,...,0,0,0,0,0,0,0,0,1,1
3,1259,0,0,119.0,189.76,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1628,1,4,120.0,174.60,9,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [12]:
df_fully_funded = df[df['Project Current Status Coded'] == 1]
df_expired = df[df['Project Current Status Coded'] == 0]
df_fully_funded = resample(df_fully_funded, n_samples=df_expired.shape[0], replace=False, random_state=0)
df_balance = df_fully_funded.append(df_expired)

In [13]:
Y = df_balance['Project Current Status Coded']
X = df_balance
del X['Project Current Status Coded']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [14]:
X_train.shape, X_test.shape

((179156, 140), (76782, 140))

In [15]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
# clf = RandomForestClassifier(random_state=0).fit(X_train, y_train)

/Users/xinwen/opt/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
clf.score(X_test, y_test)

0.647638769503269

In [17]:
y_test.value_counts(normalize=True)

0    0.501094
1    0.498906
Name: Project Current Status Coded, dtype: float64

In [19]:
len(df2.columns)

86

In [22]:
dump(clf, 'logit.joblib')

['logit.joblib']

In [2]:
clf = load('logit.joblib')

In [24]:
clf.score(X_test, y_test)

0.647638769503269

In [5]:
clf.coef_

array([[ 0.00034537,  0.02106429,  0.01693223,  0.00366984, -0.00087404,
         0.00346693, -0.25166271, -0.05981351, -0.02707468, -0.02687656,
        -0.03827362, -0.02639298,  0.02549677,  0.00320127,  0.00607152,
         0.01884536, -0.0006789 ,  0.00232383,  0.00188457,  0.01334889,
        -0.00078331,  0.00433077, -0.00283785, -0.00601775,  0.00592285,
         0.00620288,  0.01326817,  0.00041355,  0.02378477,  0.02378477,
         0.00284785, -0.00171867,  0.00051407,  0.00099059,  0.00657183,
         0.00179959,  0.00090946,  0.00125704, -0.19139181, -0.17403924,
         0.07403894,  0.06301674,  0.04563415,  0.03726532,  0.02715105,
         0.02379834,  0.00579701,  0.02301083, -0.00973028,  0.0159957 ,
         0.01568835,  0.01719021,  0.01298282,  0.01024035,  0.00030884,
         0.01199924,  0.02847537, -0.03258168, -0.04419166, -0.01620368,
        -0.02483013, -0.08758573,  0.003867  ,  0.01864482, -0.01066426,
        -0.00405161,  0.08891123, -0.11808149, -0.0

In [7]:
clf

AttributeError: 'LogisticRegression' object has no attribute 'columns'